# Biscuit Manufacturing

## Utils imports

In [5]:
from Utils import CSPRequirements as cspr
from Utils import dataImport as di

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
roll = di.build_roll(di.importDefects("defects.csv"), 501)

# Biscuit thresholds
biscuit_thresholds = {
    0: {'size': 4, 'a': 4, 'b': 2, 'c': 3},
    1: {'size': 8, 'a': 5, 'b': 4, 'c': 4},
    2: {'size': 2, 'a': 1, 'b': 2, 'c': 1},
    3: {'size': 5, 'a': 2, 'b': 3, 'c': 2}
}

# Initialize roll and defect data
roll = di.build_roll(di.importDefects("defects.csv"), 500)
problem = cspr.BiscuitProblem(name="BiscuitOptimization", rollLength=500, defect_data=roll, biscuit_thresholds=biscuit_thresholds)

print(problem.roll)

problem.assignBiscuitToSpot(0, 0)
problem.assignBiscuitToSpot(1, 3)
problem.assignBiscuitToSpot(4, 2)

print(problem.roll)

for var in problem.variables:
    print(var.state, end=", ")

print("")
print(problem.biscuit_start_pos)

print(problem.constraints[0].check())

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non